# Notebook 1: Dataset merging
<br>
Dataset:
The dataset describing the GPS coordinates of 536 taxis collected over 25 days in San Francisco (USA) is avaliable at <a href="https://crawdad.org/epfl/mobility/20090224/">https://crawdad.org/epfl/mobility/20090224/</a>.


<br>
<br>
In this notebook, the 536 datasets describing the trips of as many Taxi are merged. 


In [1]:
import pandas as pd
import glob
from datetime import timedelta
from datetime import datetime
from my_utils import *

C:\Users\giuli\Anaconda3\envs\skmob\lib\site-packages\tqdm\std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
mydateparser = lambda x: pd.to_datetime(x, unit='s') + timedelta(minutes=-7*60)

src_path = '../data/cabspottingdata/new_*.txt'
file_list = glob.glob(src_path)

In [3]:
%%time
#merge together all the datasets

dfs = []

for f in file_list:  
    df = pd.read_csv(f, sep=' ', header=None, names=['latitude', 'longitude', 'occupancy', 'time'])
    taxi_id = f.split('\\new_')[1].split('.')[0]
    df['taxi_id'] = taxi_id
    df = df[['taxi_id','latitude','longitude','occupancy','time']]
    dfs.append(df)

df_taxicab = pd.concat(dfs)

#convert time (from UTC to Local time for San Francisco)
correct_time = pd.to_datetime(df_taxicab['time'], unit='s') + timedelta(minutes=-7*60)
df_taxicab['time'] = correct_time

df_taxicab = df_taxicab.sort_values(by=['taxi_id', 'time'])

Wall time: 11 s


In [4]:
#let's have a look at the merged dataset

df_taxicab[:3]

,taxi_id,latitude,longitude,occupancy,time
23494,abboip,37.75153,-122.39447,0,2008-05-17 07:12:10
23493,abboip,37.75149,-122.39447,0,2008-05-17 07:13:34
23492,abboip,37.75149,-122.39447,0,2008-05-17 07:14:34


Each row in the dataset describes the position (GPS point) of a taxi at a specific time; each row has five attributes:
- `taxi_id`: the identifier of the taxi
- `latitude`: the latitude of the GPS point
- `longitude`: the longitude of the GPS point
- `occupancy`: specifies if the taxi has a fare or is vacant (0 = free, 1 = occupied)
- `time`: the timestamp relative at the GPS point



In [5]:
#some statistics

n_taxis = len(df_taxicab['taxi_id'].unique())
from_ = df_taxicab['time'].min()
to_ = df_taxicab['time'].max()

print("Dataset info:")
print("# of taxis: "+str(n_taxis))
print("# of GPS points: "+str(len(df_taxicab)))
print("from: "+str(from_))
print("to: "+str(to_))

Dataset info:
# of taxis: 539
# of GPS points: 11258862
from: 2008-05-17 03:00:04
to: 2008-06-10 02:25:34


In [6]:
%%time

#save the dataset
save_csv_zipped(df_taxicab,'taxicab_merged','datasets')

Wall time: 1min
